__TESTING__

In [1]:
from tensorflow.keras.models import load_model
import numpy as np 
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!ls "/content/gdrive/My Drive/Smart Composer"

my_last_decoder_model.h5  my_last_model.h5  train.pickle


In [0]:
my_decoder_model =  "/content/gdrive/My Drive/Smart Composer/my_last_decoder_model.h5"
my_encoder_model = "/content/gdrive/My Drive/Smart Composer/my_last_model.h5"
my_data = "/content/gdrive/My Drive/Smart Composer/train.pickle"

In [0]:
import pickle
with open(my_data, 'rb') as f:
    x_tr, y_tr, x_val,y_val,x_tokenizer,y_tokenizer,max_inp_len,max_out_len = pickle.load(f)

In [0]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [11]:
encoder_model = load_model(my_encoder_model, custom_objects={'AttentionLayer': AttentionLayer})

In [12]:
decoder_model = load_model(my_decoder_model, custom_objects={'AttentionLayer': AttentionLayer})

In [13]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 21, 300)           2807100   
_________________________________________________________________
lstm (LSTM)                  [(None, 21, 300), (None,  721200    
Total params: 3,528,300
Trainable params: 3,528,300
Non-trainable params: 0
_________________________________________________________________


In [14]:
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    2658600     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 300)]        0                                            
____________________________________________________________________________________________

In [0]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index
input_word_index= x_tokenizer.word_index

In [0]:
#THIS function predicts the next sentence
def bring_my_sentence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sos']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eos'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eos'  or len(decoded_sentence.split()) >= (max_out_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [0]:
#converting tensor vector to sentence
def seq2output(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sos']) and i!=target_word_index['eos']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString
#converting input tensor to sentence
def seq2input(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=input_word_index['sos']) and i!=input_word_index['eos']):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [18]:
bring_my_sentence(x_tr[3].reshape(1,max_inp_len))

" r is being sent out by bill's office today"

__what is bleu score?__
# two references for one document
from nltk.translate.bleu_score import corpus_bleu<br>
references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']]]<br>
candidates = [['this', 'is', 'a', 'test']]<br>
score = corpus_bleu(references, candidates)<br>
print(score)<br>
1.0<br>
1.0 is the perfect bleu score which means that you sentences matches the any of the sentence in corpus <br>
so i make corpus of output and store in reference and pass the predicted output in canidate to calculate the score to check whether my composer compose the right message.


In [0]:
def calculate_bleu(original,tokenize_sentence):
  references = [original]
  candidates = tokenize_sentence
  score = sentence_bleu(references, candidates)
  return score

In [20]:
output = []  
for i in range(40,50):
  pred = bring_my_sentence(x_val[i].reshape(1,max_inp_len))
  score = calculate_bleu(word_tokenize(seq2output(y_val[i])),word_tokenize(pred))
  print('----------------------------------------------------------')
  print('input seq :',seq2input(x_val[i]))
  print("Predicted seq :",pred)
  print("BLEU SCORE: ",score)
  print("----------------------------------------------------------")
  output.append({"Input seq":seq2input(x_val[i]), "Pred. Seq":pred ,"BLEU SCORE": score})
results_df = pd.DataFrame.from_dict(output) 



----------------------------------------------------------
input seq : please see the following attachments for 
Predicted seq :  participant revisions
BLEU SCORE:  1.0
----------------------------------------------------------
----------------------------------------------------------
input seq : attached are my accomplishments for this 
Predicted seq :  review period
BLEU SCORE:  1.0
----------------------------------------------------------
----------------------------------------------------------
input seq : larry attached is the current estimate 
Predicted seq :  for the remediation sites in your area
BLEU SCORE:  1.0
----------------------------------------------------------
----------------------------------------------------------
input seq : sue could you ple 
Predicted seq :  ase send me your cell phone number
BLEU SCORE:  1.0
----------------------------------------------------------
----------------------------------------------------------
input seq : the enclosed file co

In [21]:
pd.set_option('display.max_colwidth', -1)
results_df

,Input seq,Pred. Seq,BLEU SCORE
0,please see the following attachments for,participant revisions,1.000000
1,attached are my accomplishments for this,review period,1.000000
2,larry attached is the current estimate,for the remediation sites in your area,1.000000
3,sue could you ple,ase send me your cell phone number,1.000000
4,the enclosed file contains the m,onthtodate schedule c items for your viewing,1.000000
5,this,is our last game,0.428882
6,dinner this week,i'm free monwed,1.000000
7,i want to bid for,kyle brady,1.000000
8,the new,deal s are,1.000000
9,attached please find the following,documents thanks lydiax,1.000000


__END__